p i e
les 79 73 0.11493222498688235
bus 91 32 0.7023128649185882
us 92 24 0.6564355750390566


In [29]:
D = "USpowerGrid"
N = 500

EDGE_WEIGHT = 30

In [30]:
pref = {
    "angular_resolution": 1,
    "aspect_ratio": 1,
    "crossing_angle": 1,
    "crossing_number": 1,
    "gabriel_graph_property": 1,
    "ideal_edge_length": 1,
    "node_resolution": 1,
    "neighborhood_preservation": 3,
    "runtime": 1,
    "stress": 1,
}

In [31]:
# Standard Library
import argparse
from time import perf_counter

# Third Party Library
import matplotlib.pyplot as plt
import networkx as nx
import optuna
import pandas as pd
from egraph import Drawing, all_sources_bfs
from ex_utils.config.dataset import dataset_names
from ex_utils.config.paths import get_dataset_path
from ex_utils.config.quality_metrics import qm_names
from ex_utils.share import draw_and_measure, ex_path, generate_base_df_data
from ex_utils.utils.graph import (
    egraph_graph,
    load_nx_graph,
    nx_graph_preprocessing,
)
from tqdm import tqdm

In [32]:
db_uri = (
    f"sqlite:///{ex_path.joinpath('results/optimization/optimization.db')}"
)
study_name = f"{D}_N={N}_with_runtime"
study = optuna.load_study(study_name=study_name, storage=db_uri)
best_trials = study.best_trials

In [33]:
best_trials_quality_metrics = dict((qm_name, []) for qm_name in qm_names)
for best_trial in best_trials:
    for qm_name, value in zip(qm_names, best_trial.values):
        best_trials_quality_metrics[qm_name].append(value)

for qm_name in qm_names:
    best_trials_quality_metrics[qm_name] = sorted(
        best_trials_quality_metrics[qm_name]
    )

for best_trial in best_trials:
    for qm_name, value in zip(qm_names, best_trial.values):
        best_trial.set_user_attr(
            f"{qm_name}_asc_index",
            best_trials_quality_metrics[qm_name].index(value),
        )
    s = 0
    for qm_name in qm_names:
        s += best_trial.user_attrs[f"{qm_name}_asc_index"] * pref[qm_name]
    best_trial.set_user_attr("asc_index_total", s)

In [34]:
list_best_trials = []
for best_trial in best_trials:
    item = {
        "params": best_trial.params,
        "values": best_trial.values,
        **best_trial.user_attrs,
    }

    list_best_trials.append(item)

desc_sorted_list_best_trials = sorted(
    list_best_trials, key=lambda x: x["asc_index_total"], reverse=True
)

In [35]:
qm_names

['angular_resolution',
 'aspect_ratio',
 'crossing_angle',
 'crossing_number',
 'gabriel_graph_property',
 'ideal_edge_length',
 'neighborhood_preservation',
 'node_resolution',
 'runtime',
 'stress']

In [36]:
qs = []

In [37]:
dataset_path = get_dataset_path(D)
nx_graph = nx_graph_preprocessing(
    load_nx_graph(dataset_path=dataset_path), EDGE_WEIGHT
)

eg_graph, eg_indices = egraph_graph(nx_graph=nx_graph)
eg_distance_matrix = all_sources_bfs(eg_graph, EDGE_WEIGHT)

i = 0
for t in desc_sorted_list_best_trials:
    pivots = t["params"]["pivots"]
    iterations = t["params"]["iterations"]
    eps = t["params"]["eps"]
    print(pivots, iterations, eps)
    break

    eg_drawing = Drawing.initial_placement(eg_graph)

    quality_metrics = {}
    # print(len(t["values"]))

    for qm_name, value in zip(
        [
            "angular_resolution",
            "aspect_ratio",
            "crossing_angle",
            "crossing_number",
            "gabriel_graph_property",
            "ideal_edge_length",
            "neighborhood_preservation",
            "node_resolution",
            "stress",
            "runtime",
        ],
        t["values"],
    ):
        # print(qm_name, value)
        quality_metrics[qm_name] = value

    # print(quality_metrics)

    # for qm_name in qm_names:
    # t.user
    # params, quality_metrics, pos = draw_and_measure(
    #     pivots=pivots,
    #     iterations=iterations,
    #     eps=eps,
    #     eg_graph=eg_graph,
    #     eg_indices=eg_indices,
    #     eg_drawing=eg_drawing,
    #     eg_distance_matrix=eg_distance_matrix,
    #     edge_weight=EDGE_WEIGHT,
    #     seed=0,
    # )
    #     fig, ax = plt.subplots(dpi=300, facecolor="white")
    #     ax.set_aspect("equal")

    #     ax.set_title(
    #         f"""{D} pivots={pivots},iter={iterations},eps={round(eps, 4)}
    # runtime={t['values'][-1]}
    # best_n={len(best_trials)},asc_index_total={t['asc_index_total']}
    # {'''
    # '''.join([f'{qm_name}={t[f"{qm_name}_asc_index"]}' for qm_name in qm_names])}"""
    #     )

    # nx.draw(
    #     nx_graph,
    #     pos=pos,
    #     node_size=5,
    #     node_color="#AB47BC",
    #     edge_color="#CFD8DC",
    #     ax=ax,
    # )

    # print(quality_metrics)
    qs.append(quality_metrics)

    # plt.show()
    if i == 46:
        print(pivots, iterations, eps)
    i += 1

92 24 0.6564355750390566


In [38]:
from ex_utils.share import ex_path, generate_sscalers, generate_mmscalers

n_split = 10
data_seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
df_paths = [
    ex_path.joinpath(f"data/grid/{D}/seed={data_seed}_n={n_split}.pkl")
    for data_seed in data_seeds
]
sscalers = generate_sscalers(df_paths)
mmscalers = generate_mmscalers(df_paths)

In [39]:
import numpy as np

mm = []
ma = 0
i = 0
mi = 0
for q in qs:
    ssq = {}
    ssq_sum = 0
    for qm_name in qm_names:
        ssq[qm_name] = sscalers[qm_name].transform(
            np.array(q[qm_name]).reshape(-1, 1)
        )[0][0]
        ssq_sum += ssq[qm_name]
    print(ssq_sum, ssq)
    mm.append(ssq_sum)
    if ssq_sum > ma:
        ma = ssq_sum
        mi = i
    i += 1

In [40]:
mi

0

In [41]:
ma

0

In [42]:
print(sum(mm) / len(mm))

ZeroDivisionError: division by zero